In [1]:
import requests
from collections import Counter
import pandas as pd
import json

In [2]:
def get_data(study_id):
    response = requests.get(
        " https://ega-archive.org/metadata/v2/samples?queryBy=study&queryId={}&limit=0".format(study_id)
    )
    if response.status_code != 200:
        print("Study {} got error code {}".format(study_id, response.status_code))
        return []
    return response.json()['response']["result"]


def get_study_list(filename):
    with open(filename) as fi:
        study_list = list(fi.readlines())
    study_list = list(map(lambda x: x.strip(), study_list))
    return study_list
    

def main():
    
    
    rows = []
    for s in study_list:
        rows.append(get_study_info(s))

    pd.DataFrame(rows).to_csv('EGA_data.csv',sep=';')

In [3]:
study_list = get_study_list("/media/victoria/VICTORIA_EXTERNAL_DISK/BioHackathon2021/EGA_examples/EGA_studies_list.txt")

In [ ]:
study_list[1]

In [ ]:
attributes= dict()
for s in study_list:
    out = dict()
    print(s)
    results = get_data(s)
    
    out[s] = results
    ega_file = open('/media/victoria/VICTORIA_EXTERNAL_DISK/BioHackathon2021/EGA_examples/metadata_files/'+ s+ "_metadata.json", "w")
    json.dump(out, ega_file)
    ega_file.close()
    
    gender_d = dict()
    phenotype_d = dict()
    attributes_d=dict()

    for i in range(0,len(results)):
        g = results[i]['gender']
        p = results[i]['phenotype']
        a = results[i]['attributes']
        #print(a)
        if g in gender_d.keys():
            gender_d[g] += 1
        else:
            gender_d[g] = 1

        if p in phenotype_d.keys(): 
            phenotype_d[p] += 1
        else: 
            phenotype_d[p] = 1

        if a in attributes_d.keys(): 
            attributes_d[p] += 1
        else: 
            attributes_d[p] = 1
    gender_d['total'] = sum(gender_d.values())
    phenotype_d['total'] = sum(phenotype_d.values())
    attributes_d['total'] = sum(attributes_d.values())
    attributes[s] = {'gender' : gender_d, 'phenotype': phenotype_d, 'attributes': attributes_d}

EGAS00001002783
EGAS00001002784
EGAS00001002786
EGAS00001002787
EGAS00001002788
EGAS00001002789
EGAS00001002793
EGAS00001002790


In [ ]:
a_file = open("ega_gender_phenotypes_attributes_summary.json", "w")

json.dump(attributes, a_file)

a_file.close()